## Segundo Trabalho de Inteligência Artificial e Sistemas Inteligentes


Rebeca Cecco de Oliveira

In [1]:
from dinoAI import *

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_validate, GridSearchCV, RepeatedStratifiedKFold
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.base import BaseEstimator, ClassifierMixin

In [11]:
!pip install pygame

In [4]:
def improved_KeyClassifier():
    def __init__(self, state):
        self.state = state

    def keySelector(self, distance, speed, obType):
        self.state = sorted (self.state, key = first)
    for s,d in self.state:
       if speed < s:
          limDist = d
          break
    if distance <= limDist:
       if isinstance(obType, Bird):
           return "K_DOWN"
       else:
           return "K_UP"
    return "K_NO"

    def updateState(self, state):
        self.state = state

### Implementação do KM Centroids do T1

In [5]:
class KMeansCentroidsClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, estimator='kmeans', argv=None, n_clusters=8):
        self._estimator = estimator
        self._n_clusters = n_clusters
        self._argv = argv

        if estimator == 'kmeans':
          self._est = KMeans(n_clusters)
        else:
          raise Exception('Estimator not definied!')

    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = np.unique(y)

        self.X_ = X
        self.y_ = y

        # Geting centroids with estimator
        self.__groups = [self._est.fit(X[y == c]).cluster_centers_ for c in self.classes_]

        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
        X = check_array(X)

        predicted = [self.classes_[np.argmin([min([np.linalg.norm(x - c) for c in g]) for g in self.__groups])] for x in X]
        return np.array(predicted)

    def get_params(self, deep=False):
        return {'estimator': self._estimator,
                'n_clusters': self._n_clusters,
                'argv': self._argv}

### Implementação da MetaHeurística Escolida

In [6]:
class GA():
    def __init__(self, n_clusters, argv):
        self.k_ = n_clusters
        self.pop_size_ = argv['pop_size']
        self.iter_max_ = argv['iter_max']
        self.cross_ratio_ = argv['cross_ratio']
        self.mut_ratio_ = argv['mut_ratio']
        self.max_time_ = argv['max_time']

    def fit(self, X):
        # Check that X have correct shape
        X = check_array(X)

        self.X_ = X

        # Geting centroids with estimator
        solution = self.genetic(X, self.k_, self.pop_size_, self.iter_max_, 
                                       self.cross_ratio_, self.mut_ratio_, self.max_time_)
        self.cluster_centers_ = solution['cluster_centers_'],
        self.sse_ = solution['sse_'],
        self.clusters_ = solution['clusters_']

        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
        X = check_array(X)

        predicted = [self.classes_[np.argmin([np.linalg.norm(x - g) for g in self.__solution['cluster_centers_']])] for x in X]
        return np.array(predicted)


    def __available_closest_items(self, cluster, available_items, n):
        # n items more closet to centroid of cluster
        n = min(n, len(available_items))
        evaluations = [self.__evaluate_cluster(cluster + [i]) for i in available_items]
        closest = np.argpartition([e['sum_dist'] for e in evaluations], range(n))[:n]
        return closest

    def hill_climbing(self, items, k):
        num_best = k
        available_items = list(range(len(items)))
        np.random.shuffle(available_items)
        # choose the firsts items of clusters randomly
        clusters = [[items[available_items.pop()]] for _ in range(k)]

        current_cluster = 0
        while available_items:
            closest = self.__available_closest_items(clusters[current_cluster], [items[p] for p in available_items], num_best)
            choiced = available_items[closest[np.random.randint(len(closest))]]
            clusters[current_cluster].append(items[choiced])
            available_items.remove(choiced)
            current_cluster = (current_cluster + 1) % k

        return clusters

    def __evaluate_cluster(self, cluster):
        # euclidean norm
        items = [s['coord'] for s in cluster]
        mu = np.average(items, axis=0) # centroid
        
        norm2 = [np.linalg.norm(i-mu) for i in items]
        return {'sum_dist': sum(norm2), 'dist': norm2, 'mu': mu.tolist(), 'items': cluster}

    def __evaluate_clusters(self, clusters):
        states = [self.__evaluate_cluster(cluster) for cluster in clusters]
        return states

    def __objective_function(self, states):
        # SSE metric
        sse = sum([state['sum_dist'] for state in states])
        return sse

    def __random_state(self, states):
        n = len(states)-1
        if n <= 1:
            return states[0]

        index = np.random.randint(0, n)
        return states[index]

    def __initial_population(self, n, items, k):
        pop = [self.__evaluate_clusters(self.hill_climbing(items, k)) for _ in range(n)]
        return pop

    def __selection(self, population, n, n_tournament):
        # by tournament
        N = len(population)
        if n >= N:
            return population
        if n_tournament > N:
            n_tournament = N

        selecteds = np.random.choice(range(0, len(population)), size=n_tournament, replace=False)
        selecteds.sort()
        values = [self.__objective_function(population[i]) for i in selecteds]
        pos_ordered = np.argpartition(values, range(n))[-n:]
        return [population[p] for p in pos_ordered]

    def __crossover(self, dad, mom):
        # son product of an egg fertilized by a sperm
        sperm = self.__random_state(dad)['items']
        # take all groups that not contain the sperm, and remove it too
        egg = [e for e in [[n for n in m['items'] if n not in sperm] for m in mom] if e]
        son = egg + [sperm]

        k = len(dad)
        while len(son) > k: # concatenate smaller groups
            smallers = np.argpartition([len(s) for s in son], range(2))[:2]
            son[smallers[0]] += son[smallers[1]]
            del son[smallers[1]]

        while len(son) < k: # separate larger groups
            larger = np.argmax([len(s) for s in son])
            half = len(son[larger])//2
            son += [son[larger][:half]] + [son[larger][half:]]
            del son[larger]

        return self.__evaluate_clusters(son)

    def __mutation(self, items, k):
        # new individual
        return self.__evaluate_clusters(self.hill_climbing(items, k))

    def __convergent(self, population):
        clusters_0 = [sorted([i['id'] for i in p['items']]) for p in population[0]]
        clusters = [[sorted([i['id'] for i in clusters['items']]) for clusters in states] for states in population]
        return all(c == clusters_0 for c in clusters)

    def __evaluate_population(self, population):
        return sum([self.__objective_function(p) for p in population], [])

    def __offspring(self, population, n):
        best_index = np.argpartition([sum([q['sum_dist'] for q in p]) for p in population], range(n))[:n]
        return [population[i] for i in best_index]

    def genetic(self, X, k, pop_size, iter_max, cross_ratio, mut_ratio, max_time):
        items = [{'id': x, 'coord': y} for x, y in zip(range(len(X)), X)]
        n_tournament = 3
        half_pop = pop_size//2
        pop = self.__initial_population(pop_size, items, k)
        iter = 0
        end = 0

        start = time.process_time()
        while True:
            new_pop = pop.copy()
            for _ in range(half_pop): # everyone can cross
                if np.random.uniform(0, 1, 1) <= cross_ratio:
                    parents = self.__selection(pop, 2, n_tournament)
                    new_pop.append(self.__crossover(parents[0], parents[1]))
                if np.random.uniform(0, 1, 1) <= mut_ratio:
                    new_pop.append(self.__mutation(items, k))
            pop = self.__offspring(new_pop, pop_size)
            val_pop = self.__evaluate_population(pop)

            iter += 1
            end = time.process_time()
            if iter >= iter_max:
                br = 'iteration'
                break
            if end-start > max_time:
                br = 'time'
                break
            if self.__convergent(pop):
                br = 'convergence'
                break

        best_individual = self.__offspring(pop, 1)[0]
        return {'cluster_centers_': [b['mu'] for b in best_individual],
                'sse_': [b['sum_dist'] for b in best_individual],
                'clusters_': [[c['coord'] for c in b['items']] for b in best_individual]}

## Execução

In [3]:
main()

KeyboardInterrupt: 

### Resultados da Execução do Professor

In [ ]:
placares_prof = [3237, 3217, 3230, 3300, 4136, 3224, 3225, 3229, 3238, 4046, 3006, 3237, 3280, 3280, 3284, 3251, 3232, 3277, 3217, 3278, 3272, 3234, 3229, 3397, 3015, 3270, 3251, 2713, 3216, 3371]
media_prof = 3279.733333333333
std_prof = 248.07954817401256
media_menos_desvio_prof = 3031.6537851593207